In [ ]:
# requirements
# Инициализация
from langchain.chat_models.gigachat import GigaChat
from langchain.schema import HumanMessage, SystemMessage

# устанавливаем локаль
import locale
locale.setlocale(locale.LC_ALL, ('ru', 'utf-8'))
print(locale.getlocale())

# Загрузка документов
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter
)

# Эмбеддинги
from chromadb.config import Settings
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GigaChatEmbeddings

# QnA цепочка
from langchain.chains import RetrievalQA
from langchain.prompts.chat import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
token = os.getenv("token")
data_file = "./data/bzd.docx"

In [ ]:
llm = GigaChat(
    credentials=token,
    scope="GIGACHAT_API_PERS",
    model="GigaChat-Pro",
    verify_ssl_certs=False)

In [ ]:
messages = [
    SystemMessage(
        content="Ты эмпатичный бот-психолог, который помогает пользователю решить его проблемы."
    )
]

user_input = input("User: ")
messages.append(HumanMessage(content=user_input))
res = llm(messages)
messages.append(res)
# Ответ сервиса
print("Bot: ", res.content)

In [ ]:
loader = Docx2txtLoader(data_file)
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
documents = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

In [ ]:
embeddings = GigaChatEmbeddings(
    one_by_one_mode=True,
    _debug_delay=0.005,
    credentials=token,
    scope="GIGACHAT_API_PERS",
    verify_ssl_certs=False
)

db = Chroma.from_documents(
    documents,
    embeddings,
    client_settings=Settings(anonymized_telemetry=True),
)

retriever = db.as_retriever(k=4)

In [ ]:
question = "Устойчивость работы хозяйственного объекта определяется по"

qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)
print(qa_chain({"query": question}))

In [ ]:
question = "Аварией считается утечка нефти в щбьеме"

qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)
print(qa_chain({"query": question}))

In [ ]:
question = "Пожарная техника в зависимости от способа пожаротушения подразделяется на"

qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)
print(qa_chain({"query": question}))

In [ ]:
question = "Сколько классов опасных производственных объектов существует"

qa_chain = RetrievalQA.from_chain_type(llm, retriever=db.as_retriever())
print(qa_chain({"query": question}))

In [ ]:
question = "Кто такой джастин бибер?"

qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)
print(qa_chain({ "query": question}))

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
sys_templ = '''Ответь на вопрос пользователя. \
Используй при этом только информацию из контекста. Если в контексте нет \
информации для ответа, скажи "Я не знаю".
<context>
{context}
</context>
'''
qa_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            sys_templ,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
combine_docs_chain = create_stuff_documents_chain(llm, qa_prompt)

from langchain_core.messages import HumanMessage

question = "Кто такой джастин бибер?"
question1 = "Устойчивость работы хозяйственного объекта определяется по"

# Поиск по базе данных
docs = retriever.invoke(question1)

result = combine_docs_chain.invoke(
    {
        "context": docs,
        "messages": [
            HumanMessage(content=question)
        ],
    }
)

print(result)